In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = "https://raw.githubusercontent.com/DSEI21000-S21/project-tennis-ml/main/wta_matches/wta_matches-2000-2021_expanded.csv"

def readData(path):
    df = pd.read_csv(path)
    return df

def dropGibberishRows(dataframe, column_name, filter_variable):
    index_list = list(dataframe[dataframe[column_name] == filter_variable].index)
    dataframe = dataframe.drop(index_list)
    return dataframe

def encodeColumn(dataframe, column_list = []):
    df = pd.get_dummies(dataframe, columns= column_list)
    return df

def convertDate(dataframe, date):
    dataframe[date] = pd.to_datetime(dataframe[date], format = '%Y%m%d', errors='coerce')
    return dataframe

def addDateFeatures(dataframe, date):
    dataframe['year'] = dataframe[date].dt.year
    dataframe['month'] = dataframe[date].dt.month
    dataframe['day'] = dataframe[date].dt.day
    return dataframe

def appendTarget(dataframe, winner, loser):
    dataframe[winner] = 1
    dataframe[loser] = 0
    return dataframe

def getMissingDataPercentage(dataframe):
    percent_missing = dataframe.isnull().sum() * 100 / len(dataframe)
    missing_value_df = pd.DataFrame({'column_name': dataframe.columns,
                                 'percent_missing': percent_missing})
    pd.set_option("max_rows", None)
    return missing_value_df

def formatColumns(dataframe):
    columns = list(dataframe.columns)
    renamed_columns = [i.lower().strip().replace(' ', "_").replace(':','') for i in columns]
    renamed_columns_dict = {columns[i]: renamed_columns[i] for i in range(len(columns))}
    dataframe = dataframe.rename(columns = renamed_columns_dict)
    return dataframe

def convertColumnsToNumeric(dataframe, columns = []):
    dataframe[columns] = dataframe[columns].apply(pd.to_numeric, errors ='coerce')
    return dataframe 

def fillMissingDataMode(dataframe, cols):
    dataframe[cols]= dataframe[cols].fillna(dataframe[cols].mode().iloc[0])
    return dataframe

def dropColumn(dataframe, column_list = []):
    return dataframe.drop(column_list, axis =1)

def fillMissingDataMean(dataframe, cols):
    dataframe[cols]= dataframe[cols].fillna(dataframe[cols].mean().iloc[0])
    return dataframe

def fillMissingData999(dataframe, cols):
    dataframe[cols]= dataframe[cols].fillna(999)
    return dataframe

In [14]:
wta_data = readData(path)
wta_data = convertDate(wta_data, 'tourney_date')
wta_data = addDateFeatures(wta_data, 'tourney_date')
mask = ((wta_data['year']== 2020) & (wta_data['tourney_name'].str.contains("Roland")))
wta_data['french_open_2020'] = np.where(mask, 1, 0)
wta_data = dropColumn(wta_data, ['player_entry', 'tourney_name'])
wta_data = fillMissingDataMode(wta_data, ['surface', 'player_hand'])
wta_data = fillMissingDataMean(wta_data, ['player_height', 'player_age', 'minutes', 'ace', 'double_fault', 'service_points_won', 'first_serve_made', 'first_serve_won', 
'second_serve_won', 'serve_game', 'break_point_saved', 'break_point_faced', 'player_rank','player_rank_points'])
wta_data = fillMissingData999(wta_data, ['player_seed'])
wta_data = dropColumn(wta_data, ['player_ioc', 'score', 'tourney_id', 'tourney_date', 'player_name'])
wta_data = encodeColumn(wta_data, ['surface', 'tourney_level', 'player_hand','round'])
wta_data = formatColumns(wta_data)

In [17]:
getMissingDataPercentage(wta_data)

,column_name,percent_missing
draw_size,draw_size,0.0
match_num,match_num,0.0
player_id,player_id,0.0
player_seed,player_seed,0.0
player_height,player_height,0.0
player_age,player_age,0.0
best_of,best_of,0.0
minutes,minutes,0.0
ace,ace,0.0
double_fault,double_fault,0.0


In [18]:
wta_data.info(verbose =True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119438 entries, 0 to 119437
Data columns (total 55 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   draw_size           119438 non-null  int64  
 1   match_num           119438 non-null  int64  
 2   player_id           119438 non-null  int64  
 3   player_seed         119438 non-null  float64
 4   player_height       119438 non-null  float64
 5   player_age          119438 non-null  float64
 6   best_of             119438 non-null  int64  
 7   minutes             119438 non-null  float64
 8   ace                 119438 non-null  float64
 9   double_fault        119438 non-null  float64
 10  service_points_won  119438 non-null  float64
 11  first_serve_made    119438 non-null  float64
 12  first_serve_won     119438 non-null  float64
 13  second_serve_won    119438 non-null  float64
 14  serve_game          119438 non-null  float64
 15  break_point_saved   119438 non-nul

In [19]:
wta_data.describe()

,draw_size,match_num,player_id,player_seed,player_height,player_age,best_of,minutes,ace,double_fault,...,player_hand_u,round_br,round_f,round_qf,round_r128,round_r16,round_r32,round_r64,round_rr,round_sf
count,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,...,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000,119438.000000
mean,54.576600,82.883705,202298.029580,678.641488,173.385424,24.219136,3.002579,155.925050,72.479200,73.225291,...,0.039694,0.000017,0.021501,0.084395,0.111723,0.168573,0.314540,0.143171,0.113013,0.043068
std,41.109025,171.380133,3351.576382,463.529275,5.000963,4.647098,0.071770,37.753336,84.053933,83.532933,...,0.195241,0.004092,0.145047,0.277981,0.315027,0.374376,0.464334,0.350248,0.316610,0.203012
min,2.000000,1.000000,200001.000000,1.000000,153.000000,14.039699,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,9.000000,201212.000000,11.000000,173.000000,21.122519,3.000000,173.385424,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,32.000000,25.000000,201450.000000,999.000000,173.385424,23.956194,3.000000,173.385424,5.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,64.000000,92.000000,201619.000000,999.000000,174.000000,26.964408,3.000000,173.385424,173.385424,173.385424,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,128.000000,2701.000000,223140.000000,999.000000,189.000000,173.385424,5.000000,2475.000000,173.385424,173.385424,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
wta_data.to_pickle("./wta_cleaned_data.pkl")

In [21]:
wta_data = pd.read_pickle("./wta_cleaned_data.pkl")
wta_data.head()

,draw_size,match_num,player_id,player_seed,player_height,player_age,best_of,minutes,ace,double_fault,...,player_hand_u,round_br,round_f,round_qf,round_r128,round_r16,round_r32,round_r64,round_rr,round_sf
0,4,1,201419,999.0,172.000000,18.173854,3,173.385424,173.385424,173.385424,...,0,0,0,0,0,0,0,0,1,0
1,4,2,200085,999.0,163.000000,24.821355,3,173.385424,173.385424,173.385424,...,0,0,0,0,0,0,0,0,1,0
2,4,1,200652,999.0,173.385424,26.973306,3,173.385424,173.385424,173.385424,...,0,0,0,0,0,0,0,0,1,0
3,4,2,200128,999.0,189.000000,24.457221,3,173.385424,173.385424,173.385424,...,0,0,0,0,0,0,0,0,1,0
4,4,1,200017,999.0,173.385424,28.928131,3,173.385424,173.385424,173.385424,...,0,0,0,0,0,0,0,0,1,0


In [22]:
wta_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119438 entries, 0 to 119437
Data columns (total 55 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   draw_size           119438 non-null  int64  
 1   match_num           119438 non-null  int64  
 2   player_id           119438 non-null  int64  
 3   player_seed         119438 non-null  float64
 4   player_height       119438 non-null  float64
 5   player_age          119438 non-null  float64
 6   best_of             119438 non-null  int64  
 7   minutes             119438 non-null  float64
 8   ace                 119438 non-null  float64
 9   double_fault        119438 non-null  float64
 10  service_points_won  119438 non-null  float64
 11  first_serve_made    119438 non-null  float64
 12  first_serve_won     119438 non-null  float64
 13  second_serve_won    119438 non-null  float64
 14  serve_game          119438 non-null  float64
 15  break_point_saved   119438 non-nul